# Introduction

We want our model to estimate election outcomes based on the fundamentals and poll data. Thus, we need a "poll feature" for each district upon which to train. Unfortunately, many districts do not have poll data. We overcome this by interpolating poll results for districts that do not have polls from districts that do have polls based on districts' similarity in demographics and historical election results.

The poll data points upon which we will interpolate are [538](https://fivethirtyeight.com/features/2018-house-forecast-methodology/)'s adjusted poll margins. We will pull the most recent adjusted margin (MRAM).

We perform the interpolation using [radial basis function interpolation](http://num.math.uni-goettingen.de/schaback/teaching/sc.pdf) (RBF interpolation).

In [1]:
# !pip install git+git://github.com/treverhines/RBF@master
    
import pandas as pd
import numpy as np
import scipy.cluster
import scipy.interpolate
from scipy import stats
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae
# from rbf.interpolate import RBFInterpolant
# import rbf.basis

from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers

import matplotlib.pyplot as plt
plt.style.use('ggplot')

print('done')

Using TensorFlow backend.


done


# Input Data

We will now interpolate real polls based on a large range of input features. We predict based on ProximityOne's [demographic data by congressional district](http://proximityone.com/cd.htm). Specifically, we input the [social characteristics](http://proximityone.com/cd161dp2.htm).

In [ ]:
df = pd.read_excel('all.xlsx', sheet='Sheet 1')

ins = ['S' + str(rep).zfill(3) for rep in range(1, df.shape[1])]
outs = ['MRAM']

print('done')

done


# Preprocessing

We obviously must drop all non-numeric data. However, we also must drop rows in which coordinates are duplicated due to quirks in the way the RBF interpolation is calculated (duplicate coordinates result in `scipy` trying to invert a singular matrix).

In [ ]:
# Drop rows that have non-numerical data
df.dropna(inplace=True)
df = df[df.applymap(np.isreal).any(1)]
df.columns = ['MRAM'] + ins

# Drop rows that down have MRAM
df = df[df['MRAM'] != 999]

inp = Input(shape=(517,))
encoded = Dense(517, activation='relu',
               activity_regularizer=regularizers.l1(10e-5))(inp)
encoded = Dense(400, activation='relu',
               activity_regularizer=regularizers.l1(10e-5))(encoded)
encoded = Dense(200, activation='relu',
               activity_regularizer=regularizers.l1(10e-5))(encoded)
encoded = Dense(100, activation='relu',
               activity_regularizer=regularizers.l1(10e-5))(encoded)
encoded = Dense(50, activation='relu',
               activity_regularizer=regularizers.l1(10e-5))(encoded)

decoded = Dense(100, activation='relu')(encoded)
decoded = Dense(200, activation='relu')(decoded)
decoded = Dense(400, activation='relu')(decoded)
decoded = Dense(517, activation='sigmoid')(decoded)

autoencoder = Model(inp, decoded)
print('starting')
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

# encoded_input = Input(shape=(515,))
# decoder_layer = autoencoder.layers[-1]
# decoder = Model(encoded_input, decoder_layer(encoded_input))

# tidx = [np.random.randint(0, len(df)) for i in range(int(5 * len(df) / 10))]
# vidx = list(set(range(515)).difference(set(tidx)))

tidx = range(0, int(len(df) / 2))
vidx = range(int(len(df) / 2), len(df))

trows = [df.iloc[idx] for idx in tidx]
vrows = [df.iloc[idx] for idx in vidx]
df_train = df.drop(df.index[tidx])
df_validate = df.drop(df.index[vidx])

            
autoencoder.fit(df.as_matrix(), df.as_matrix(),
                epochs=1000,
                batch_size=256,
                shuffle=True)
            


# def get_repeat_idxs(records_array):
#     idx_sort = np.argsort(records_array)
#     sorted_records_array = records_array[idx_sort]
#     vals, idx_start, count = np.unique(sorted_records_array, return_counts=True,
#                                     return_index=True)
#     res = np.split(idx_sort, idx_start[1:])
#     vals = vals[count > 1]
#     return filter(lambda x: x.size > 1, res)

# # Drop data columns that have repeats
# bads = []
# for cni in range(len(ins)):
#     cn = ins[cni]
#     col = np.array(df[cn])
#     repeated = list(get_repeat_idxs(col))
    
#     if len(repeated) > 0:
#         bads.append(cn)

# for cn in bads:
#     ins.remove(cn)
#     df.drop(cn, axis=1, inplace=True)
    
# # Drop rows with repeats
# for name in ins + outs:
#     _, uidx = np.unique(df[name], return_index=True)
#     df = df.iloc[uidx]

# df.reset_index(drop=True, inplace=True)
# df.to_csv('tmp.csv', sep=',')

# print('done')

starting
Epoch 1/1000
106/106 [==============================] - 0s 3ms/step - loss: 38025531392.0000
Epoch 2/1000
106/106 [==============================] - 0s 498us/step - loss: 38025293824.0000
Epoch 3/1000
106/106 [==============================] - 0s 533us/step - loss: 38025011200.0000
Epoch 4/1000
106/106 [==============================] - 0s 523us/step - loss: 38024900608.0000
Epoch 5/1000
106/106 [==============================] - 0s 694us/step - loss: 38024974336.0000
Epoch 6/1000
106/106 [==============================] - 0s 606us/step - loss: 38024851456.0000
Epoch 7/1000
106/106 [==============================] - 0s 493us/step - loss: 38024810496.0000
Epoch 8/1000
106/106 [==============================] - 0s 491us/step - loss: 38024757248.0000
Epoch 9/1000
106/106 [==============================] - 0s 506us/step - loss: 38024720384.0000
Epoch 10/1000
106/106 [==============================] - 0s 590us/step - loss: 38024704000.0000
Epoch 11/1000
106/106 [===================

Epoch 87/1000
106/106 [==============================] - 0s 636us/step - loss: 38024617984.0000
Epoch 88/1000
106/106 [==============================] - 0s 721us/step - loss: 38024617984.0000
Epoch 89/1000
106/106 [==============================] - 0s 730us/step - loss: 38024617984.0000
Epoch 90/1000
106/106 [==============================] - 0s 679us/step - loss: 38024622080.0000
Epoch 91/1000
106/106 [==============================] - 0s 695us/step - loss: 38024617984.0000
Epoch 92/1000
106/106 [==============================] - 0s 692us/step - loss: 38024622080.0000
Epoch 93/1000
106/106 [==============================] - 0s 714us/step - loss: 38024617984.0000
Epoch 94/1000
106/106 [==============================] - 0s 537us/step - loss: 38024626176.0000
Epoch 95/1000
106/106 [==============================] - 0s 527us/step - loss: 38024622080.0000
Epoch 96/1000
106/106 [==============================] - 0s 480us/step - loss: 38024622080.0000
Epoch 97/1000
106/106 [=================

Epoch 172/1000
106/106 [==============================] - 0s 580us/step - loss: 38024617984.0000
Epoch 173/1000
106/106 [==============================] - 0s 914us/step - loss: 38024622080.0000
Epoch 174/1000
106/106 [==============================] - 0s 756us/step - loss: 38024622080.0000
Epoch 175/1000
106/106 [==============================] - 0s 593us/step - loss: 38024622080.0000
Epoch 176/1000
106/106 [==============================] - 0s 690us/step - loss: 38024617984.0000
Epoch 177/1000
106/106 [==============================] - 0s 685us/step - loss: 38024613888.0000
Epoch 178/1000
106/106 [==============================] - 0s 652us/step - loss: 38024617984.0000
Epoch 179/1000
106/106 [==============================] - 0s 658us/step - loss: 38024617984.0000
Epoch 180/1000
106/106 [==============================] - 0s 527us/step - loss: 38024622080.0000
Epoch 181/1000
106/106 [==============================] - 0s 644us/step - loss: 38024622080.0000
Epoch 182/1000
106/106 [======

Epoch 257/1000
106/106 [==============================] - 0s 605us/step - loss: 38024613888.0000
Epoch 258/1000
106/106 [==============================] - 0s 538us/step - loss: 38024622080.0000
Epoch 259/1000
106/106 [==============================] - 0s 511us/step - loss: 38024622080.0000
Epoch 260/1000
106/106 [==============================] - 0s 547us/step - loss: 38024613888.0000
Epoch 261/1000
106/106 [==============================] - 0s 562us/step - loss: 38024622080.0000
Epoch 262/1000
106/106 [==============================] - 0s 429us/step - loss: 38024613888.0000
Epoch 263/1000
106/106 [==============================] - 0s 520us/step - loss: 38024613888.0000
Epoch 264/1000
106/106 [==============================] - 0s 482us/step - loss: 38024622080.0000
Epoch 265/1000
106/106 [==============================] - 0s 537us/step - loss: 38024617984.0000
Epoch 266/1000
106/106 [==============================] - 0s 669us/step - loss: 38024617984.0000
Epoch 267/1000
106/106 [======

Epoch 342/1000
106/106 [==============================] - 0s 608us/step - loss: 38024617984.0000
Epoch 343/1000
106/106 [==============================] - 0s 547us/step - loss: 38024622080.0000
Epoch 344/1000
106/106 [==============================] - 0s 568us/step - loss: 38024622080.0000
Epoch 345/1000
106/106 [==============================] - 0s 512us/step - loss: 38024617984.0000
Epoch 346/1000
106/106 [==============================] - 0s 839us/step - loss: 38024613888.0000
Epoch 347/1000
106/106 [==============================] - 0s 676us/step - loss: 38024617984.0000
Epoch 348/1000
106/106 [==============================] - 0s 535us/step - loss: 38024622080.0000
Epoch 349/1000
106/106 [==============================] - 0s 652us/step - loss: 38024622080.0000
Epoch 350/1000
106/106 [==============================] - 0s 769us/step - loss: 38024622080.0000
Epoch 351/1000
106/106 [==============================] - 0s 728us/step - loss: 38024622080.0000
Epoch 352/1000
106/106 [======

Epoch 427/1000
106/106 [==============================] - 0s 580us/step - loss: 38024622080.0000
Epoch 428/1000
106/106 [==============================] - 0s 504us/step - loss: 38024617984.0000
Epoch 429/1000
106/106 [==============================] - 0s 593us/step - loss: 38024617984.0000
Epoch 430/1000
106/106 [==============================] - 0s 611us/step - loss: 38024622080.0000
Epoch 431/1000
106/106 [==============================] - 0s 453us/step - loss: 38024622080.0000
Epoch 432/1000
106/106 [==============================] - 0s 442us/step - loss: 38024622080.0000
Epoch 433/1000
106/106 [==============================] - 0s 573us/step - loss: 38024622080.0000
Epoch 434/1000
106/106 [==============================] - 0s 429us/step - loss: 38024617984.0000
Epoch 435/1000
106/106 [==============================] - 0s 498us/step - loss: 38024622080.0000
Epoch 436/1000
106/106 [==============================] - 0s 419us/step - loss: 38024617984.0000
Epoch 437/1000
106/106 [======

Epoch 512/1000
106/106 [==============================] - 0s 506us/step - loss: 38024613888.0000
Epoch 513/1000
106/106 [==============================] - 0s 527us/step - loss: 38024617984.0000
Epoch 514/1000
106/106 [==============================] - 0s 697us/step - loss: 38024622080.0000
Epoch 515/1000
106/106 [==============================] - 0s 671us/step - loss: 38024622080.0000
Epoch 516/1000
106/106 [==============================] - 0s 478us/step - loss: 38024622080.0000
Epoch 517/1000
106/106 [==============================] - 0s 667us/step - loss: 38024622080.0000
Epoch 518/1000
106/106 [==============================] - 0s 687us/step - loss: 38024617984.0000
Epoch 519/1000
106/106 [==============================] - 0s 716us/step - loss: 38024617984.0000
Epoch 520/1000
106/106 [==============================] - 0s 643us/step - loss: 38024617984.0000
Epoch 521/1000
106/106 [==============================] - 0s 697us/step - loss: 38024617984.0000
Epoch 522/1000
106/106 [======

Epoch 597/1000
106/106 [==============================] - 0s 595us/step - loss: 38024617984.0000
Epoch 598/1000
106/106 [==============================] - 0s 455us/step - loss: 38024622080.0000
Epoch 599/1000
106/106 [==============================] - 0s 436us/step - loss: 38024617984.0000
Epoch 600/1000
106/106 [==============================] - 0s 446us/step - loss: 38024617984.0000
Epoch 601/1000
106/106 [==============================] - 0s 424us/step - loss: 38024622080.0000
Epoch 602/1000
106/106 [==============================] - 0s 432us/step - loss: 38024617984.0000
Epoch 603/1000
106/106 [==============================] - 0s 401us/step - loss: 38024613888.0000
Epoch 604/1000
106/106 [==============================] - 0s 538us/step - loss: 38024617984.0000
Epoch 605/1000
106/106 [==============================] - 0s 393us/step - loss: 38024622080.0000
Epoch 606/1000
106/106 [==============================] - 0s 400us/step - loss: 38024622080.0000
Epoch 607/1000
106/106 [======

# Training

In [ ]:
def gen_interpolator(df_train, _ins, _outs):
    inrep = [np.array(df_train[rep]).astype(float) for rep in _ins]
    outrep = [np.array(df_train[rep]).astype(float) for rep in _outs]
    features = list(inrep) + list(outrep)
    
    # Check for repeats (there shouldn't be any)
    for feature in features:
        if len(feature) != len(set(feature)):
            print('Repeat')
    return scipy.interpolate.Rbf(*features)

# Validation

We partition our data into training and validation sets. We train the interpolation on most of the available data points, leaving out a set of validation points. We then predict the *MRAM* of the validation points and compare to the true *MRAM*, calculating the error in our interpolation.

**TODO**: get the correct number of training points corresponding to the number of available polls

In [ ]:
# Validate interpolation on many partitions
totcor = 0
y_pred_all = []
y_true_all = []
N = 1
for k in range(N):
#     tidx = [np.random.randint(0, len(df)) for i in range(int(5 * len(df) / 10))]
    tidx = range(85)
    trows = [df.iloc[idx] for idx in tidx]
    df_train = df.drop(df.index[tidx])
    
    
    inrep = [np.array(df_train[rep]).astype(float) for rep in ins]
    outrep = [np.array(df_train[rep]).astype(float) for rep in outs]
    features = list(inrep) + list(outrep)
    
    # Check for repeats (there shouldn't be any)
    for feature in features:
        if len(feature) != len(set(feature)):
            print('Repeat')
            
            
            
    rbfi = scipy.interpolate.Rbf(*features)
    
    
    
    
    
#     rbfi = gen_interpolator(df_train, ins, outs)
    
    y_pred = [float(rbfi(*[row[rep] for rep in ins])) for row in trows]
    y_true = [row[outs[0]] for row in trows]

    correct = []
    for i in range(len(y_pred)):
        correct.append((y_pred[i] > 0.5) == (y_true[i] > 0.5))

    totcor += sum(correct) / len(y_pred)
    y_pred_all.extend(y_pred)
    y_true_all.extend(correct)

# This represents the expected rate at which the interpolator correctly predicts the outcome of the race
print('Fraction of Races Predicted Correctly: ' + str(totcor / N))
print('Mean Squared Prediction Error: ' + str(mse(y_true_all, y_pred_all)))
print('Mean Absolute Prediction Error: ' + str(mae(y_true_all, y_pred_all)))
print('Mean Prediction Error: ' + str(np.mean(np.array(y_true_all) - np.array(y_pred_all))))
print('Stdev Prediction Error: ' + str(np.std(np.array(y_true_all) - np.array(y_pred_all))))

plt.hist(np.array(y_true_all) - np.array(y_pred_all))
plt.title('y_true - y_pred')
plt.show()

print('done')

In [ ]:
res = stats.probplot(np.array(y_true_all) - np.array(y_pred_all), plot=plt)

print('done')

# Final Answer

We now plug in all available data to an interpolator and interpolate all rows. Keep in mind we expect our interpolation error to be drawn from the distribution shown above.

In [ ]:
# TODO do it